In [ ]:
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split
import time
import random
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def process_dataset(ary):
    n_event = ary[:,1]
    inic_q_avg = ary[:, 2]
    hit_signal = ary[:, 3]

    out = []
    for item in [n_event, hit_signal, inic_q_avg]:
        out.append(convert_np_array(item))
        
    return out

In [ ]:
def get_input(ary):
    init_hit = ary[0]
    signal = process_signal(ary[1])
    #signal = ary[1]
    
    inic_q_avg = ary[2]
    return init_hit, signal, inic_q_avg

In [ ]:
def process_signal(ary, normalization = False, flatten = False, pad = 1):
    if normalization:
            ary = np.array([i.reshape(4,4,1)/np.sum(i) for i in ary])
    else:
        ary = np.array([i.reshape(4,4,1) for i in ary])
    if flatten:
            ary = np.array([i.reshape(16) for i in ary])
    if pad:
        ary = np.pad(ary[:, :, :, :], ((0, 0), (pad, pad), (pad, pad), (0,0)), 'constant')
    return ary

In [17]:
def convert_np_array(ary):
    return np.array([i for i in ary])

In [19]:
# New photon
def get_charge(folder = "./Data/Run4_MC_photon_signal/", side = 'A'):
    output = []
    start = time.time()
    for i in range(1000, 3000, 1000):
        if i % 5000 == 0:
            print("event", i, "time", time.time() - start)
            start = time.time()
        try:
            output.append(pd.read_pickle(folder + f"{side}_RPD_photon{i}.pickle"))
        except FileNotFoundError:
            print(f"Missing file: {i}")
            continue
    return pd.concat(output, ignore_index = True).set_index('Event number').astype(float)

In [ ]:
def get_unit_vector(arr):
    output = []
    norm =  np.linalg.norm(arr, axis = 1)
    for i in range(arr.shape[0]):
        output.append([arr[i][0] / norm[i], arr[i][1] / norm[i]])
    return np.array(output)
        

In [20]:
path = "/home/shengy3/ML_RPD/Data/Toy_fermi_qq_fiber_021022"
A_charge = get_charge(side = 'A',folder =  f"{path}/pickle/").drop_duplicates()
B_charge = get_charge(side = 'B',folder =  f"{path}/pickle/").drop_duplicates()

In [21]:
A_charge.to_pickle(f"{path}/Acharge.pickle")
B_charge.to_pickle(f"{path}/Bcharge.pickle")


In [22]:
del_list = []
for i in (A_charge.index):
    if i not in B_charge.index:
        print(i)
        del_list.append(i)
        
A_charge = A_charge.drop(del_list)

In [23]:
test_size = 0.4
trainA, validA, trainB, validB = train_test_split(A_charge, B_charge, test_size=test_size, random_state = 42)

In [24]:
#validA.to_pickle("./Data/ToyV1_Fermi_2.7TeV_Merge_Charge_122420/validA.pickle")
#validB.to_pickle("./Data/ToyV1_Fermi_2.7TeV_Merge_Charge_122420/validB.pickle")

validA.to_pickle(f"{path}/validA_dict.pickle")
validB.to_pickle(f"{path}/validB_dict.pickle")



#np.save("./Data/ToyV1_Fermi_2.7TeV_Merge_Charge_122420/validA.npy", validA)
#np.save("./Data/ToyV1_Fermi_2.7TeV_Merge_Charge_122420/validB.npy", validB)



In [25]:
train = trainA.append(trainB).to_numpy()
valid = validA.append(validB).to_numpy()

In [26]:
train_signal = train[:,8:]
valid_signal = valid[:,8:]

In [30]:
t_inic_q_avg = train[:, 2:4]
v_inic_q_avg = valid[:, 2:4]

In [31]:
t_unit_vector = get_unit_vector(t_inic_q_avg)
v_unit_vector = get_unit_vector(v_inic_q_avg)

In [32]:
t_signal = process_signal(train_signal, pad = 1, normalization = False)
v_signal = process_signal(valid_signal, pad = 1, normalization = False)

# Training

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
plt.rcParams.update({"savefig.bbox": 'tight'})
from tensorflow.keras.models import Sequential
import tensorflow.keras as keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, Reshape
from tensorflow.keras.models import Model
import tensorflow
from sklearn.model_selection import train_test_split
import os
import time
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.backend as K


In [34]:
def gaussian_nll(ytrue, ypreds):
    """Keras implmementation of multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]
    
    mse = -0.5*K.sum(K.square((ytrue-mu)/K.exp(logsigma)),axis=1)
    sigma_trace = -K.sum(logsigma, axis=1)
    log2pi = -0.5*n_dims*np.log(2*np.pi)
    
    log_likelihood = mse+sigma_trace+log2pi

    return K.mean(-log_likelihood)

In [56]:
# define two sets of inputs
# n_neutron = Input(shape=(1,), name = 'interact_neutron') # (40, ta_n)
# # the first branch operates on the first input
# x = Dense(4, activation="relu")(n_neutron)
# x = Dense(4, activation="relu")(x)
# x = Model(inputs=n_neutron, outputs=x)


# the second branch opreates on the second input
bias = Input(shape=(6,6,1), name = 'biased_RPD')
y = Conv2D(filters = 64, kernel_size = (2,2),\
           padding = 'Same', activation ='relu')(bias)
y = Conv2D(filters = 64, kernel_size = (2,2),\
           padding = 'Same', activation ='relu')(y)
y = Flatten()(y)

y = Dense(32, activation = "relu")(y)
y = Dense(16, activation = "relu")(y)
y = Dense(8, activation = "relu")(y)
    
y = Model(inputs=bias, outputs=y)




Q_avg = Dense(2, activation="linear", name = 'Q_avg')(y.output)

model = Model(inputs=[y.input], outputs=[Q_avg])


In [57]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics = ['mae', 'mse']
)

In [58]:
model.fit(
    { 'biased_RPD': t_signal},
    #{"Q_avg": t_inic_q_avg, "Q_std": box_size},
    { "Q_avg": t_unit_vector},
    
    epochs= 75,
    batch_size=10000,
    validation_data = ({ 'biased_RPD': v_signal},\
                       {"Q_avg": v_unit_vector})
)

Train on 2400 samples, validate on 1602 samples
Epoch 1/75
2400/2400 [==============================] - 1s 389us/sample - loss: 1731.0371 - mae: 33.3462 - mse: 1731.0371 - val_loss: 7101.9858 - val_mae: 75.1197 - val_mse: 7101.9858
Epoch 2/75
2400/2400 [==============================] - 0s 160us/sample - loss: 6910.1118 - mae: 74.0911 - mse: 6910.1118 - val_loss: 1587.5669 - val_mae: 36.1095 - val_mse: 1587.5669
Epoch 3/75
2400/2400 [==============================] - 0s 160us/sample - loss: 1541.3818 - mae: 35.6016 - mse: 1541.3818 - val_loss: 1171.7874 - val_mae: 31.6490 - val_mse: 1171.7874
Epoch 4/75
2400/2400 [==============================] - 0s 162us/sample - loss: 1143.2883 - mae: 31.2268 - mse: 1143.2883 - val_loss: 386.7155 - val_mae: 15.5474 - val_mse: 386.7155
Epoch 5/75
2400/2400 [==============================] - 0s 161us/sample - loss: 375.2444 - mae: 15.3089 - mse: 375.2444 - val_loss: 689.9193 - val_mae: 24.3428 - val_mse: 689.9193
Epoch 6/75
2400/2400 [================

In [59]:
# model.save(f'./Output/Model/V1Fermi_Run4MC_photon_020222.h5')

In [61]:
QA = model.predict({ 'biased_RPD': t_signal.astype(float)})


# Evaluation  

In [85]:
def get_unit_vector(arr):
    output = []
    norm =  np.linalg.norm(arr, axis = 1)
    for i in range(arr.shape[0]):
        output.append([arr[i][0] / norm[i], arr[i][1] / norm[i]])
    return np.array(output)
        

In [86]:
def correct_ang(angA, angB):
    output = []
    for i in range(angA.shape[0]):
        diff = angA[i] - angB[i]
        if abs(diff) > np.pi:
            min_diff = min(abs(diff + 2 * np.pi), abs(diff - 2 * np.pi))
            diff = min_diff if diff < 0 else -min_diff
        output.append(diff)
    return np.array(output)

In [63]:
validA = np.load(f"{path}/validA_dict.pickle", allow_pickle=True)
validB = np.load(f"{path}/validB_dict.pickle", allow_pickle=True)

In [64]:
np_A = validA.to_numpy()
np_B = validB.to_numpy()

A_signal = process_signal(np_A[:, 8:])
B_signal = process_signal(np_B[:, 8:])

QA = model.predict([A_signal.astype(float)])
QB = model.predict([B_signal.astype(float)])

A_inic_q_avg = np_A[:, 2:4].astype(float)
B_inic_q_avg = np_B[:, 2:4].astype(float)

A_hit = np_A[:, 5]
B_hit = np_B[:, 5]

pt_nuclear = np_A[:, 1]

In [66]:
dQA  = A_inic_q_avg.astype(float) - QA
dQB  = B_inic_q_avg.astype(float)  - QB

Adx = dQA[:,0]
Ady = dQA[:,1]

Bdx = dQB[:,0]
Bdy = dQB[:,1]

Apsi_rec = np.arctan2(QA[:,1],QA[:,0])
Apsi_gen = np.arctan2(A_inic_q_avg[:,1],A_inic_q_avg[:,0])
AR_rec = np.sqrt(QA[:,0] ** 2 + QA[:, 1] ** 2)


Bpsi_rec = np.arctan2(QB[:,1],QB[:,0])
Bpsi_gen = np.arctan2(B_inic_q_avg[:,1],B_inic_q_avg[:,0])
BR_rec = np.sqrt(QB[:,0] ** 2 + QB[:, 1] ** 2)

In [67]:
#A_inic_q_avg = get_unit_vector(A_inic_q_avg)
#B_inic_q_avg = get_unit_vector(B_inic_q_avg)

In [68]:
def average_vector(QA, QB):
    NormA = np.linalg.norm(QA, axis = 1)
    NormB = np.linalg.norm(QB, axis = 1)
    NA = np.array([QA[:,0 ] / NormA, QA[:,1 ] / NormA])
    NB = np.array([QB[:,0 ] / NormB, QB[:,1 ] / NormB])
    flip_B = -NB
    avgx = (NA[0] + flip_B[0]) / 2     
    avgy = (NA[1] + flip_B[1]) / 2     
    avg = np.arctan2(avgy, avgx)    
    return NA, NB, avgx, avgy, avg

In [69]:
NA, NB, avg_recon_x, avg_recon_y, avg_recon_angle = average_vector(QA, QB)

In [70]:
NA_gen, NB_gen, avg_gen_x, avg_gen_y, avg_gen_angle = average_vector(A_inic_q_avg, B_inic_q_avg)

In [71]:

Apsi_gen = np.arctan2(A_inic_q_avg[:,1],A_inic_q_avg[:,0])
Apsi_true = validA.iloc[:,0].astype(float).to_numpy()


Bpsi_gen = np.arctan2(B_inic_q_avg[:,1],B_inic_q_avg[:,0])
Bpsi_true = validB.iloc[:,0].astype(float).to_numpy()


In [82]:
Apsi_gen = np.arctan2(A_inic_q_avg[:,1],A_inic_q_avg[:,0])
Apsi_true = np_A[:,2].astype(float)
AX = np_A[:, 0].astype(float)
AY = np_A[:, 1].astype(float)
AR = np.sqrt(AX**2+AY**2)

Bpsi_gen = np.arctan2(B_inic_q_avg[:,1],B_inic_q_avg[:,0])
Bpsi_true = np_B[:,2].astype(float)
BX = np_B[:, 0].astype(float)
BY = np_B[:, 1].astype(float)
BR = np.sqrt(BX**2+BY**2)



In [78]:
gen_rec = correct_ang(Apsi_gen, Apsi_rec) 
true_rec = correct_ang(Apsi_true, Apsi_rec) 
true_gen = correct_ang(Apsi_true, Apsi_gen) 

# ROOT 

In [83]:
TreeA = {"n_incident_neutron": A_hit,
        "X_gen":AX,
         "Y_gen": AY,
         "R_gen": AR,
         "Qx_rec": QA[:,0],
         "Qy_rec": QA[:,1],
         "dX": Adx,
         "dY": Ady,
         "psi_true":Apsi_true,
         "psi_rec":Apsi_rec,
         "psi_gen": Apsi_gen,
         "R_rec": AR_rec}

TreeB = {"n_incident_neutron": B_hit,
         "X_gen":BX,
         "Y_gen": BY,
         "R_gen": BR,
         "Qx_rec": QB[:,0],
         "Qy_rec": QB[:,1],
         "dX": Bdx,
         "dY": Bdy,
         "psi_true":Bpsi_true,
         "psi_rec":Bpsi_rec,
         "psi_gen": Bpsi_gen,
         "R_rec": BR_rec}

Tree_arms = {"NormAx":NA[0],
             "NormAy":NA[1],
             "NormBx": -NB[0],
             "NormBy": -NB[1],
             "Average_RP_vector_X": avg_recon_x,
             "Average_RP_vector_Y": avg_recon_y,
             "Average_RP_angle": avg_recon_angle,
             "pt_nuclear": pt_nuclear
            }

In [84]:
output_folder = "./Output/fig/Performance/V1Fermi_Run4MC_qq_fiber_photon_020222/"

In [66]:
np.save(output_folder + "TreeA.npy", TreeA)
np.save(output_folder + "TreeB.npy", TreeB)
np.save(output_folder + "Tree_arms.npy", Tree_arms)